In [2]:
import hash_blocking_spark as hash
import ngram_blocking_spark as ngram
import matchers_spark as match
import similarity_spark as sim
import blocking_structured_and_sorted_spark as ss
import cluster_spark as cluster

Beispiel nehme unser Match, cluster mit Spark und schreibe das Ergebnis von Spark als CSV

In [5]:
import pandas as pd
def reconstructed_pairs(path):
    df_pairs = pd.read_csv(path)
    return list(zip(df_pairs['dblp_index'], df_pairs['acm_index']))

pairs = reconstructed_pairs('../Matched/Matched Entities.csv')

In [7]:
import csv
def cluster_to_csv(cluster_data, filename):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for idx, item in enumerate(cluster_data, start=1):
            writer.writerow([f'Cluster {idx}: {item}'])

cluster_pairs = cluster.build_clusters_parallel(pairs)
cluster_to_csv(cluster_pairs, '../Matched/Spark_Cluster.csv')
print(len(cluster_pairs))

934


In [8]:
import pandas as pd
dblp_csv = '../CSV-files/dblp_stem.csv'
dblp = pd.read_csv(dblp_csv)

acm_csv = '../CSV-files/acm_stem.csv'
acm = pd.read_csv(acm_csv)


In [9]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sparkk").getOrCreate()

acm_df = spark.read.csv(acm_csv, header=True, inferSchema=True)
dblp_df = spark.read.csv(dblp_csv, header=True, inferSchema=True)

selected_columns = ['author_names', 'paper_title']
c = ngram.n_gram_blocking(acm_df, selected_columns,2)
d = ngram.n_gram_blocking(dblp_df, selected_columns,2)
e = match.apply_ngram_blocks_spark(c,d, 0.7, sim.jaccard_similarity_ngrams)


spark.stop

Processing time: 13.989503860473633 seconds. Number of index combinations: 0


<bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x0000021E98713FD0>>

In [10]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sparkk").getOrCreate()

acm_df = spark.read.csv(acm_csv, header=True, inferSchema=True)
dblp_df = spark.read.csv(dblp_csv, header=True, inferSchema=True)

selected_columns = ['author_names', 'paper_title']
c = ngram.initial_ngram_parallel_df(spark,acm_df, selected_columns,2)
d = ngram.initial_ngram_parallel_df(spark, dblp_df, selected_columns,2)
e = match.apply_similarity_blocks_spark(c,d, 0.7, sim.jaccard_similarity)
#e = apply_similarity_blocks_spark(c, d, 0.1, sim.jaccard_similarity_ngrams)

spark.stop
#db = ngram.initial_ngram_parallel(dblp_csv, selected_columns, 2)

#simo = match.apply_similarity_blocks_spark(ap, db, 0.7, hash_indices, sim.jaccard_similarity_ngrams)


ValueError: Cannot convert column into bool: please use '&' for 'and', '|' for 'or', '~' for 'not' when building DataFrame boolean expressions.

In [11]:
from itertools import product

# Your data
data = [
    {'index1': '[53908b4920f70186a0dbacb7]', 'index2': '[53e9a297b7602d9702b9cf03]', 'similarity': '1.0'},
    {'index1': '[5390958a20f70186a0df0138]', 'index2': '[53e9af2db7602d9703969dc1]', 'similarity': '1.0'},
    {'index1': '[53908a9620f70186a0da4d9c]', 'index2': '[53e9b98ab7602d970457bd03]', 'similarity': '1.0'},
    {'index1': '[539087f320f70186a0d6f876, 53908b4920f70186a0dbb083]', 'index2': '[53e9ad04b7602d97036e13e5]', 'similarity': '1.0'},
    # Add more rows here
]

# Initialize a list to store processed rows
processed_data = []

# Iterate through each row
for row in data:
    # Split the index values and remove brackets
    index1_values = row['index1'][1:-1].split(', ')
    index2_values = row['index2'][1:-1].split(', ')

    # Check if either index has multiple values
    if len(index1_values) > 1 or len(index2_values) > 1:
        # Generate combinations of indices
        index_combinations = list(product(index1_values, index2_values))

        # Create a new row for each combination
        for combination in index_combinations:
            processed_row = {
                'index1': combination[0],
                'index2': combination[1],
                'similarity': row['similarity']
            }
            processed_data.append(processed_row)
    else:
        # If no index has multiple values, simply append the row
        processed_data.append(row)

# Print the processed data
for row in processed_data:
    print(row)


{'index1': '[53908b4920f70186a0dbacb7]', 'index2': '[53e9a297b7602d9702b9cf03]', 'similarity': '1.0'}
{'index1': '[5390958a20f70186a0df0138]', 'index2': '[53e9af2db7602d9703969dc1]', 'similarity': '1.0'}
{'index1': '[53908a9620f70186a0da4d9c]', 'index2': '[53e9b98ab7602d970457bd03]', 'similarity': '1.0'}
{'index1': '539087f320f70186a0d6f876', 'index2': '53e9ad04b7602d97036e13e5', 'similarity': '1.0'}
{'index1': '53908b4920f70186a0dbb083', 'index2': '53e9ad04b7602d97036e13e5', 'similarity': '1.0'}


In [12]:
from itertools import product

# Define your pairs
pairs = [
    {'index1': [1], 'index2': [3]},
    {'index1': [2], 'index2': [4]}

]

combined_pairs = []

# Iterate through each pair
for pair in pairs:
    # Generate combinations of index1 and index2 values
    combinations = list(product(pair['index1'], pair['index2']))

    # Append combinations to the combined_pairs list
    combined_pairs.extend(combinations)

print(combined_pairs)



[(1, 3), (2, 4)]


In [13]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import time
from pyspark.sql.types import StringType
from pyspark.sql.types import StringType, ArrayType, StructField, StructType
def apply_similarity_blocks_spark(blocks1, blocks2, threshold, field_name, similarity_function):
    start_time = time.time()



    df1 = spark.read.csv(blocks1, header=True, inferSchema=True)
    df2 = spark.read.csv(blocks2, header=True, inferSchema=True)
    data_tuples1 = [(key, value[field_name], value['index']) for key, value in blocks1.items()]
    data_tuples2 = [(key, value[field_name], value['index']) for key, value in blocks2.items()]
    
    schema = StructType([       
        StructField("blocking_key", StringType(), nullable=False),
        StructField(field_name, StringType(), nullable=False),
        StructField("index", ArrayType(StringType()), nullable=False)
    ])
    
    blocks1_df = spark.createDataFrame(data_tuples1, schema=schema)
    blocks2_df = spark.createDataFrame(data_tuples2, schema=schema)
  
   
    similarity_udf = F.udf(lambda set1, set2: str(similarity_function(set(set1), set(set2))), StringType())

    joined_blocks = df1.alias("block1").crossJoin(df2.alias("block2"))
    similar_pairs_df = joined_blocks.where(
        (F.col("block1.value") == F.col("block2.value"))
    ).select(
        F.col("block1.index").alias("index1"),
        F.col("block2.index").alias("index2"),
        similarity_udf(F.col("block1.value"), F.col("block2.value")).alias("similarity")
    )

    similar_pairs_df = similar_pairs_df.filter(F.col("similarity") >= threshold)
    similar_pairs = similar_pairs_df.collect()
    
    index_combinations = [(i, j) for pair in similar_pairs for i in pair['index1'] for j in pair['index2']]
    index_combinations = list(set(index_combinations))

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Processing time: {elapsed_time} seconds. Number of index combinations: {len(similar_pairs)}")

    return similar_pairs

In [15]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sparkk").getOrCreate()

acm_df = spark.read.csv(acm_csv, header=True, inferSchema=True)
dblp_df = spark.read.csv(dblp_csv, header=True, inferSchema=True)

selected_columns = ['author_names', 'paper_title']
c = hash.initial_hash_parallel(acm_df, selected_columns, spark)
d = hash.initial_hash_parallel(dblp_df, selected_columns, spark)
e = match.apply_similarity_blocks_spark(c, d, 0.7, sim.jaccard_similarity)

Processing time: 7.17189359664917 seconds. Number of index combinations: 782


In [16]:
year_block = [1995,1996,1997, 1998, 1999,2000,2001, 2002, 2003, 2004,2005]
labels = ["1995", "1996", "1997", "1998", "1999", "2000", "2001", "2002", "2003", "2004"]
selected_columns = ['author_names', 'paper_title']
ap = ss.block_by_year_and_publisher(acm_csv, year_block, labels)
db = ss.block_by_year_and_publisher(dblp_csv, year_block, labels)

In [17]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sparkk").getOrCreate()

acm_df = spark.read.csv(acm_csv, header=True, inferSchema=True)
dblp_df = spark.read.csv(dblp_csv, header=True, inferSchema=True)

selected_columns = ['paper_title']
c = hash.initial_hash_parallel(acm_df, selected_columns, spark)
d = hash.initial_hash_parallel(dblp_df, selected_columns, spark)
e = match.apply_similarity_blocks_spark(c, d, 0.7, sim.jaccard_similarity)